# Synacor Challenge VM

**Solution Started**: July 7, 2025  
**Author**: Abbas Moosajee

This notebook loads and runs the Synacor challenge binary using a custom `VirtualMachine` implementation, highlights challenge codes, and displays the interactive game output in styled HTML.

In [87]:
import os, re, time, copy, json
from collections import defaultdict, deque
from html import escape
from IPython.display import display, HTML
from VirtualMachine import VirtualMachine
start_time = time.time()

In [88]:
# Load the binary program file
program_file = "challenge.bin"
file_path = os.path.join(os.getcwd(), program_file)  # assumes file in current working dir
vm_program = open(file_path, "rb").read()

In [ ]:
class SynacorConsole:
    track_backwards = {"north":"south", "south":"north", "east":"west","west":"east"}

    def __init__(self, software: bytes, spec_code: str, visualize: bool = True):
        self.software = software
        self.visualize = visualize
        self.console = self.reset_console()
        self.__extract_codes(" ", [spec_code])

    def reset_console(self):
        console = VirtualMachine(self.software)
        self.game_map = defaultdict(list)
        self.overall_commands = []
        self.challenge_codes = {}
        self.valid_codes = set()
        self.code_no = 0
        return console

    @staticmethod
    def md5_hash(code: str) -> str:
        import hashlib
        return hashlib.md5(code.encode('utf-8')).hexdigest()

    def __extract_codes(self, text, codes = []):
        """Extract mixed-case challenge codes and store them with their MD5 hashes."""
        codes_found = re.findall(r"[A-Za-z]*[a-z]+[A-Z]+[a-z]+[A-Za-z]*", text)
        codes.extend(codes_found)

        new_codes = []
        for code in codes:
            if code not in self.valid_codes:
                self.code_no += 1
                self.challenge_codes[self.code_no] = (code, self.md5_hash(code))
                new_codes.append(code)
                self.valid_codes.add(code)
        return new_codes

    def __format_terminal(self, text, code_color, input_color):
        """Highlight any detected code-like strings in the terminal."""
        lines = []
        for line in text.splitlines():
            # Simple input highlight: lines starting with '>'
            if line.strip().startswith('>>'):
                line = f"<span style='color:{input_color}'>{line}</span>"
            else:
                line = re.sub(      # Example: highlight code-like patterns
                    r"\b((?:[a-z]+[A-Z]+[a-z]|[A-Z]+[a-z]+[A-Z])\w*)\b",
                    rf"<span style='color:{code_color}'>\1</span>", line
                )
            lines.append(line)
        return "\n".join(lines)

    def display_terminal(self, text, actions):
        """Format and display terminal output with highlighted codes and interleaved actions."""
        self.color_dict = {
            "background": "#282828", "terminal": "#33FF00",
            "codes": "#FF0000", "input": "#0073FF"
        }

        # Normalize line endings and remove excessive empty lines
        text = re.sub(r'\n{3,}', '\n\n', text.strip())  # At most 2 newlines
        parts = re.split(r'(What do you do\?)', text)

        output_lines = []

        if actions:
            output_lines.append(f">> {actions[0]}")
        action_index = 1 if actions else 0

        # Now interleave remaining actions after each prompt
        i = 0
        while i < len(parts):
            output_lines.append(parts[i])
            input_prompt = "What do you do?"
            if i + 1 < len(parts) and parts[i + 1] == input_prompt:
                output_lines.append(input_prompt) # Append prompt
                if action_index < len(actions):   # Append next action if available
                    output_lines.append(f">> {actions[action_index]}")
                    action_index += 1
                i += 2
            else:
                i += 1

        combined_text = "\n".join(output_lines)
        longest_line = max(max(len(line) for line in combined_text.split('\n')), 150)

        html = (
            f"<div style='background-color: {self.color_dict['background']}; "
            f"width: {longest_line + 2}ch; padding: 0.75ex;'>"
            f"<pre style='background-color: {self.color_dict['background']}; "
            f"color: {self.color_dict['terminal']}; margin: 0; font-family: monospace; "
            f"width: {longest_line + 2}ch;'>"
            + self.__format_terminal(combined_text, self.color_dict['codes'], self.color_dict['input'])
            + "</pre></div>"
        )

        display(HTML(html))

    def play_game_manually(self, actions = []):
        """
        Game Instructions:
        - look: You may merely 'look' to examine the room, or you may 'look ' (such as 'look chair') to examine something specific.
        - go: You may 'go ' to travel in that direction (such as 'go west'), or you may merely '' (such as 'west').
        - inv: To see the contents of your inventory, merely 'inv'.
        - take: You may 'take ' (such as 'take large rock').
        - drop: To drop something in your inventory, you may 'drop '.
        - use: You may activate or otherwise apply an item with 'use '.
        """
        if actions:
            self.overall_commands.extend(actions)
        full_terminal = self.console.run_computer(actions)
        current_terminal = full_terminal[-1]
        self.display_terminal(current_terminal, actions)
        self.__extract_codes(current_terminal)
        return self.overall_commands

    def __parse_game_state(self, terminal):
        lines = terminal.splitlines()
        location, description, things, exits = ("", "", [], [])

        current_section = None
        for line_no, line in enumerate(lines):
            line = line.strip()
            if line.startswith("==") and line.endswith("=="):
                location = line.strip("= ").strip()
                description = lines[line_no + 1]
            elif line.startswith("Things of interest here:"):
                current_section = "things"
            elif line.startswith("There") and "exit" in line:
                current_section = "exits"
            elif line.startswith("-") and current_section:
                item = line[2:].strip()
                if current_section == "things":
                    things.append(item)
                elif current_section == "exits":
                    exits.append(item)
            else:
                current_section = None

        return location, description, things, exits

    def __use_items(self, item, exits):
        """Return actions to collect and use the item, and what items will be added to inventory."""

        if item == 'empty lantern':
            return [f"take {item}", f"use can", "use lantern", exits[0], exits[0], 'passage'], {"lantern"}
        else:
            return [f"take {item}", f"use {item}"], {item}

    def create_room_id(self, text: str) -> int:
        h = self.md5_hash(text.strip().lower())
        # Take first 4 hex digits, convert to int, and mod 10000 to get a 4-digit number
        return int(h[:8], 16) % 10000

    def bfs(self):
        """Perform BFS to explore and map out the game world."""
        steps = 0
        visited = set()
        room_info = {}
        inventory = set()
        item_paths = {}
        BASE_ROOM = "Foothills"
        MAX_STEPS = 250

        # Each entry: (console_state, pending_actions, path_so_far, full_action_history)
        queue = [(self.console.replicate(0), [], [], [])]

        while queue and steps < MAX_STEPS:
            steps += 1
            state, pending_actions, path, action_history = queue.pop(0)


            last_room = path[-1] if path else ""
            reverse_dir = self.track_backwards.get(last_room, "")
            current_actions = []

            # Run actions on a fresh copy of the game state
            game_copy = state.replicate(steps)
            output = game_copy.run_computer(pending_actions)
            terminal_output = output[-1]
            self.__extract_codes(terminal_output)

            # Parse game state
            room, desc, items, exits = self.__parse_game_state(terminal_output)
            room_id = self.create_room_id(desc)
            room_info[room_id] = (room, desc)
            if 'Ruins' == room:
                print(action_history)
                break
            # Skip if already visited with current inventory
            if (room_id, tuple(inventory)) in visited:
                continue
            visited.add((room_id, tuple(inventory)))

            # Handle item collection
            if items:
                item = items[0]
                item_paths[item] = action_history
                item_actions, collected = self.__use_items(item, exits)

                # Example condition: skip collecting lantern unless we have a can
                if item == "empty lantern" and "can" not in inventory:
                    continue

                inventory.update(collected)
                current_actions.extend(item_actions)
                new_history = action_history + item_actions
                action_history.extend(item_actions)
                queue.append((game_copy, item_actions, path, new_history))

            # Explore exits
            for direction in exits:
                if direction == reverse_dir and items:
                    continue  # Avoid immediate backtracking after taking item
                if desc == "You are in a dark, narrow passage." and direction == 'west':
                    path = path + [direction]
                    current_actions = current_actions + [direction]
                    action_history = action_history + [direction]
                next_path = path + [direction]
                next_actions = current_actions + [direction]
                next_history = action_history + [direction]
                queue.append((game_copy, next_actions, next_path, next_history))

        print("steps:", steps)
        # print(item_paths)
        return action_history

    def play_game(self, actions = []):

        return self.bfs()


In [90]:
console = SynacorConsole(vm_program, "LDOb7UGhTi")
# command_list = console.play_game()
test = console.bfs()

['take tablet', 'use tablet', 'doorway', 'north', 'north', 'bridge', 'continue', 'down', 'west', 'passage', 'ladder', 'west', 'south', 'north', 'take can', 'use can', 'west', 'ladder', 'cavern', 'east', 'east', 'take empty lantern', 'use can', 'use lantern', 'west', 'west', 'passage', 'darkness', 'continue', 'west', 'west', 'west', 'west']
['take tablet', 'use tablet', 'doorway', 'north', 'north', 'bridge', 'continue', 'down', 'west', 'passage', 'ladder', 'west', 'south', 'north', 'take can', 'use can', 'west', 'ladder', 'cavern', 'east', 'east', 'take empty lantern', 'use can', 'use lantern', 'west', 'west', 'passage', 'darkness', 'continue', 'west', 'west', 'west', 'west']
['take tablet', 'use tablet', 'doorway', 'north', 'north', 'bridge', 'continue', 'down', 'west', 'passage', 'ladder', 'west', 'south', 'north', 'take can', 'use can', 'west', 'ladder', 'cavern', 'east', 'east', 'take empty lantern', 'use can', 'use lantern', 'west', 'west', 'passage', 'darkness', 'continue', 'west'

In [91]:
manual_console = SynacorConsole(vm_program, "LDOb7UGhTi")
manual_console.play_game_manually()
game_actions = [
    # 'take tablet', 'use tablet', 'doorway', 'north', 'north', 'bridge', 'continue', 'down', 'west', 'passage',
    # 'ladder', 'west', 'south', 'north', 'take can', 'west', 'ladder', 'cavern', 'east', 'east', 'take empty lantern',
    # 'use can', 'use lantern', 'west', 'west', 'passage', 'darkness', 'continue', 'west', 'west',
    # "west", "west", #"north", "take red coin", "north"
    'take tablet', 'use tablet', 'doorway', 'north', 'north', 'bridge', 'continue', 'down', 'west', 'passage', 'ladder', 'west', 'south', 'north', 'take can', 'use can', 'west', 'ladder', 'cavern', 'east', 'east', 'take empty lantern', 'use can', 'use lantern', 'west', 'west', 'passage', 'darkness', 'continue', 'west', 'west', 'west', 'west', 'north', 'take red coin', 'use red coin', 'north', 'south', 'south', 'east', 'east', 'west', 'west'
    ]
all_actions = manual_console.play_game_manually(game_actions)
print(manual_console.challenge_codes)


{1: ('LDOb7UGhTi', '76ec2408e8fe3f1753c25db51efd8eb3'), 2: ('ImoFztWQCvxj', '0e6aa7be1f68d930926d72b3741a145c'), 3: ('BNCyODLfQkIl', '7997a3b2941eab92c1c0345d5747b420'), 4: ('pWDWTEfURAdS', '186f842951c0dcfe8838af1e7222b7d4'), 5: ('rdMkyZhveeIv', '2bf84e54b95ce97aefd9fc920451fc45')}


In [92]:
for code_no, code_group in console.challenge_codes.items():
    print(code_no, code_group)
print(f"Execution Time = {time.time() - start_time:.5f}s")

1 ('LDOb7UGhTi', '76ec2408e8fe3f1753c25db51efd8eb3')
2 ('ImoFztWQCvxj', '0e6aa7be1f68d930926d72b3741a145c')
3 ('BNCyODLfQkIl', '7997a3b2941eab92c1c0345d5747b420')
4 ('pWDWTEfURAdS', '186f842951c0dcfe8838af1e7222b7d4')
5 ('rdMkyZhveeIv', '2bf84e54b95ce97aefd9fc920451fc45')
Execution Time = 7.21798s
